In [ ]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it
    
from functools import partial
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne

import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray, combine
from ieeg.viz.parula import parula_map

# TODO: hmm fix these utils imports, import the funcs individually. 6/1/25.

import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label
from scipy.stats import norm
from scipy.stats import ttest_ind

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import copy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup

from src.analysis.config import experiment_conditions

from src.analysis.utils.labeled_array_utils import (
    put_data_in_labeled_array_per_roi_subject,
    remove_nans_from_labeled_array,
    remove_nans_from_all_roi_labeled_arrays,
    concatenate_conditions_by_string,
    get_data_in_time_range
)

from src.analysis.decoding.decoding import (
    concatenate_and_balance_data_for_decoding, 
    get_and_plot_confusion_matrix_for_rois_jim,
    Decoder, 
    windower,
    get_confusion_matrices_for_rois_time_window_decoding_jim,
    compute_accuracies,
    perform_time_perm_cluster_test_for_accuracies,
    plot_accuracies,
    mixup2
)

from src.analysis.spec.wavelet_functions import get_uncorrected_wavelets, get_uncorrected_multitaper, get_sig_tfr_differences, plot_mask_pages
from src.analysis.spec.subjects_tfr_objects_functions import load_or_make_subjects_tfr_objects, get_sig_tfr_differences_per_subject, get_sig_tfr_differences_per_roi

from src.analysis.utils.general_utils import (
    make_or_load_subjects_electrodes_to_ROIs_dict,
    get_good_data,
    get_sig_chans_per_subject,
    make_sig_electrodes_per_subject_and_roi_dict,
    calculate_total_electrodes,
    check_sampling_rates
)

import mne.time_frequency
from ieeg.calc.scaling import rescale
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np
from sklearn.model_selection import StratifiedKFold
from typing import Union, List, Sequence, Dict
import doctest
import gc

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)



['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\

#### 0. Load data.   
Need a way to load in the frequency information too, not just trials x channels x timepoints. Because I'm going to use frequency as a decoding feature too. For each electrode, for each training set, I think I can just mask the multitaper with the significant clusters and use that as the decoding feature, and then concatenate across electrodes to build the full training matrix.

In [ ]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
subjects = ['D0057','D0059', 'D0063', 'D0103'] # just try a few subjects cuz memory issues rn

# params - these will become input variables once i functionalize this stuff
# subjects = ['D0103']
signal_times = [-1.0, 1.5]
acc_trials_only = False
error_trials_only = False
stat_func = partial(ttest_ind, equal_var=False, nan_policy='omit')
p_thresh = 0.05
ignore_adjacency = 1 # ignore the channels dimension for clusters, just find clusters over frequency and time
n_perm = 100
n_jobs = 1
freqs = np.arange(2, 200., 2.)
n_cycles = freqs / 2
return_itc = False
time_bandwidth=10 
spec_method = 'multitaper'
average=False
seed=None
tails=2
n_splits=5
n_repeats=5
random_state=42
explained_variance=0.8
balance_method = 'subsample'
normalize='all'
obs_axs=0
chans_axs=1
oversample=True
alpha=1.
clear_memory=True

# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')
HOME = os.path.expanduser("~")

USER = os.path.basename(HOME)

if os.name == 'nt':  # Windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
elif sys.platform == 'darwin':  # macOS
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
else:  # Linux (cluster)
    # Check if we're on the cluster by looking for /cwork directory
    if os.path.exists(f"/cwork/{USER}"):
        LAB_root = f"/cwork/{USER}"
    else:
        # Fallback for other Linux systems
        LAB_root = os.path.join(HOME, "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)

task='GlobalLocal'
conditions = experiment_conditions.stimulus_big_letter_conditions # set this to whichever conditions you're running

stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = list(conditions.keys()) # get the condition names as a list

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [3]:
if conditions == experiment_conditions.stimulus_conditions:
    conditions_save_name = 'stimulus_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_experiment_conditions:
    conditions_save_name = 'stimulus_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_main_effect_conditions:
    conditions_save_name = 'stimulus_main_effect_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwpc_conditions:
    conditions_save_name = 'stimulus_lwpc_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwps_conditions:
    conditions_save_name = 'stimulus_lwps_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    conditions_save_name = 'stimulus_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    conditions_save_name = 'stimulus_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_task_conditions:
    conditions_save_name = 'stimulus_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_congruency_conditions:
    conditions_save_name = 'stimulus_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    conditions_save_name = 'stimulus_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

elif conditions == experiment_conditions.response_conditions:
    conditions_save_name = 'response_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_experiment_conditions:
    conditions_save_name = 'response_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_big_letter_conditions:
    conditions_save_name = 'response_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_small_letter_conditions:
    conditions_save_name = 'response_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_task_conditions:
    conditions_save_name = 'response_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_congruency_conditions:
    conditions_save_name = 'response_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_switch_type_conditions:
    conditions_save_name = 'response_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [4]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0103


get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [5]:
# rois_dict = {
#     'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
#     'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
#     'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
# }

# rois_dict = {
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
# }

rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}

rois = list(rois_dict.keys())
all_electrodes_per_subject_roi, sig_electrodes_per_subject_roi = make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LAS

In [6]:
print('total sig elecs:', sum(len(sig_chans_per_subject[sub]) for sub in sig_chans_per_subject))

total sig elecs: 259


get number of sig and all electrodes per subject and across subjects

In [7]:
for roi in rois:
    for sub in subjects:
        sig_elecs = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        all_elecs = all_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        print(f"Subject {sub}, ROI {roi}, Num of Sig Electrodes: {len(sig_elecs)}, Num of All Electrodes: {len(all_elecs)}")

Subject D0057, ROI lpfc, Num of Sig Electrodes: 2, Num of All Electrodes: 17
Subject D0059, ROI lpfc, Num of Sig Electrodes: 3, Num of All Electrodes: 5
Subject D0063, ROI lpfc, Num of Sig Electrodes: 7, Num of All Electrodes: 36
Subject D0103, ROI lpfc, Num of Sig Electrodes: 8, Num of All Electrodes: 11
Subject D0057, ROI occ, Num of Sig Electrodes: 1, Num of All Electrodes: 1
Subject D0059, ROI occ, Num of Sig Electrodes: 0, Num of All Electrodes: 1
Subject D0063, ROI occ, Num of Sig Electrodes: 0, Num of All Electrodes: 2
Subject D0103, ROI occ, Num of Sig Electrodes: 3, Num of All Electrodes: 4


In [8]:
# Example usage:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, all_electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

Total number of significant lpfc electrodes across all subjects: 20
Total number of lpfc electrodes across all subjects: 265
Total number of significant occ electrodes across all subjects: 4
Total number of occ electrodes across all subjects: 73


#### 1. For each electrode, make multitaper using all training trials, for both conditions to be compared (this can only do two conditions, can't do more rn)

TODO: Need to loop over the trials and use some as training set, and have some held out as a test set that isn't used to find clusters. Maybe do this in the stats step though.

In [9]:
subjects_tfr_objects = load_or_make_subjects_tfr_objects(
    layout=layout,
    spec_method=spec_method,
    conditions_save_name=conditions_save_name,
    subjects=subjects,
    conditions=conditions,
    signal_times=signal_times,
    freqs=freqs,
    n_cycles=n_cycles,
    time_bandwidth=time_bandwidth,
    return_itc=return_itc,
    n_jobs=n_jobs,
    average=average,
    acc_trials_only=acc_trials_only,
    error_trials_only=error_trials_only 
)

No existing TFR data found. Creating new file at: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\multitaper\subjects_tfr_objects\stimulus_big_letter_conditions_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_4_subjects_multitaper
Loading existing TFR object for sub-D0057, condition: Stimulus_bigS
Loading existing TFR object for sub-D0057, condition: Stimulus_bigH
Loading existing TFR object for sub-D0059, condition: Stimulus_bigS
Loading existing TFR object for sub-D0059, condition: Stimulus_bigH
Loading existing TFR object for sub-D0063, condition: Stimulus_bigS
Loading existing TFR object for sub-D0063, condition: Stimulus_bigH
Creating TFR object for sub-D0103, condition: Stimulus_bigS
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.ed

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_acq-01_space-ACPC_electrodes.tsv.


...

Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_acq-01_space-ACPC_electrodes.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Not fully anonymizing info - keeping his_id, sex, and hand info
<RawEDF | sub-D0103_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf, 225 x 7454720 (3640.0 s), ~253 kB, data not loaded>
outlier round 1 channels: ['LTMI8']
outlier round 2 channels: ['LTMI8', 'LFAM1']
outlier round 2 channels: ['LTMI8', 'LFAM1', 'LTPI1']
Reading 0 ... 5583980  =      0.000 ...  2726.553 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['1', '2', 'Response/c25.0/n25.0/BigLetters/SmallLetters/Taskg/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount337.0/BlockTrialCount1.0/ReactionTime883.0/Accuracy1.0/D103', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount339.0/BlockTrialCount3.0/ReactionTime1066.0/Accuracy1.0/D103', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse115.0/CorrectResp

c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:1621: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  all_trials = mne.concatenate_epochs(all_trials_list)


Not setting metadata
226 matching events found
No baseline correction applied
NOTE: tfr_multitaper() is a legacy function. New code should use .compute_tfr(method="multitaper").


#### 2. Find clusters that are significantly different between the two conditions, in the multitaper spectrogram  
do this for all channels per subject, and then also for all channels in an roi across subjects


In [10]:
# # THIS TAKES FOREVER DON'T RUN IT, JUST FOR DEBUGGING SO I CAN PLOT THE SIG CLUSTERS

# # For per-subject analysis (no electrode filtering needed)
# sig_elec_masks_per_subject, sig_elec_pvals_per_subject = get_sig_tfr_differences_per_subject(subjects_tfr_objects=subjects_tfr_objects, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

# all_elec_masks_per_subject, all_elec_pvals_per_subject = get_sig_tfr_differences_per_subject(subjects_tfr_objects=subjects_tfr_objects, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

# # For per-ROI analysis (with electrode filtering)
# sig_elec_masks_per_roi, sig_elec_pvals_per_roi = get_sig_tfr_differences_per_roi(subjects_tfr_objects=subjects_tfr_objects, electrodes_per_subject_roi=sig_electrodes_per_subject_roi, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

# all_elec_masks_per_roi, all_elec_pvals_per_roi = get_sig_tfr_differences_per_roi(subjects_tfr_objects=subjects_tfr_objects, electrodes_per_subject_roi=all_electrodes_per_subject_roi, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

plot these clusters

In [11]:
# # TODO: Update the mean differences to be per subject and per roi, currently copied over from wavelet_differences.
# first_sub = subjects[0]
# first_condition = list(subjects_tfr_objects[first_sub].keys())[0]
# ch_names = subjects_tfr_objects[first_sub][first_condition].ch_names
# times = subjects_tfr_objects[first_sub][first_condition].times
# freqs = subjects_tfr_objects[first_sub][first_condition].freqs

# subjects_tfr_objects_save_dir = os.path.join(layout.root, 'derivatives', 'spec', spec_method, 'subjects_tfr_objects')

# # Now plot the mask pages:
# for sub in subjects:
#     sig_elecs_mask = sig_elec_masks_per_subject[sub]
#     sig_elecs_mask_pages = plot_mask_pages(sig_elecs_mask,
#                     ch_names,
#                     times=times,
#                     freqs=freqs,
#                     channels_per_page=60,
#                     grid_shape=(6, 10),
#                     cmap=parula_map,
#                     title_prefix=f"{sub} ",
#                     log_freq=True,
#                     show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(sig_elecs_mask_pages):
#         fig_name = f"{sub}_sig_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)

#     all_elecs_mask = all_elec_masks_per_subject[sub]
#     all_elecs_mask_pages = plot_mask_pages(all_elecs_mask,
#                         ch_names,
#                         times=times,
#                         freqs=freqs,
#                         channels_per_page=60,
#                         grid_shape=(6, 10),
#                         cmap=parula_map,
#                         title_prefix=f"{sub} ",
#                         log_freq=True,
#                         show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(all_elecs_mask_pages):
#         fig_name = f"{sub}_all_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)

# for roi in rois:
#     sig_elecs_roi_mask = sig_elec_masks_per_roi[roi]
#     sig_elecs_roi_mask_pages = plot_mask_pages(sig_elecs_roi_mask,
#                     ch_names,
#                     times=times,
#                     freqs=freqs,
#                     channels_per_page=60,
#                     grid_shape=(6, 10),
#                     cmap=parula_map,
#                     title_prefix=f"{sub} {roi} ",
#                     log_freq=True,
#                     show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(sig_elecs_roi_mask_pages):
#         fig_name = f"{sub}_{roi}_sig_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)

#     all_elecs_roi_mask = all_elec_masks_per_roi[roi]
#     all_elecs_roi_mask_pages = plot_mask_pages(all_elecs_roi_mask,
#                     ch_names,
#                     times=times,
#                     freqs=freqs,
#                     channels_per_page=60,
#                     grid_shape=(6, 10),
#                     cmap=parula_map,
#                     title_prefix=f"{sub} {roi} ",
#                     log_freq=True,
#                     show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(all_elecs_roi_mask_pages):
#         fig_name = f"{sub}_{roi}_all_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)
            
# # # get the mean differences themselves and plot them
# # mean_diff_inc_vs_con = mean_diff(incongruent_spec._data, congruent_spec._data, axis=0)
# # mean_diff_switch_vs_repeat = mean_diff(switch_spec._data, repeat_spec._data, axis=0)

# # # Now, plot the mean differences directly:
# # congruency_mean_diff_pages = plot_mask_pages(
# #     mean_diff_inc_vs_con,
# #     incongruent_spec.ch_names,
# #     times=incongruent_spec.times,
# #     freqs=incongruent_spec.freqs,
# #     grid_shape=(6, 10),
# #     cmap=parula_map,  # play with color maps
# #     title_prefix=f"{sub} Mean Inc-Con Diff: ",
# #     log_freq=True,
# #     show=False
# # )

# # # Save each page as a separate figure file:
# # for i, fig in enumerate(congruency_mean_diff_pages):
# #     if rescaled:
# #         fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
# #     else:
# #         fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
# #     fig_pathname = os.path.join(save_dir, fig_name)
# #     fig.savefig(fig_pathname, bbox_inches='tight')
# #     print("Saved figure:", fig_name)

let's try creating roi labeled arrays and then masking these and decoding using them

make roi_labeled_arrays, a dict where the keys are rois and the values are LabeledArrays. Index the same way as a nested dict. Use .labels to get labels from current level.


#### roi_labeled_arrays structure
- **roi1**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **freqs**: This is the number of frequencies in the roi, each frequency is labeled as frequency name.
          - **samples**: 1 sample as a float. This is the time for this sample.

- **roi2**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **freqs**: This is the number of frequencies in the roi, each frequency is labeled as frequency name.
          - **samples**: 1 sample as a float. This is the time for this sample.

- **roiX**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **freqs**: This is the number of frequencies in the roi, each frequency is labeled as frequency name.
          - **samples**: 1 sample as a float. This is the time for this sample.


In [12]:
# Example usage:

# choose whether to use sig elecs or all elecs
electrodes = all_electrodes_per_subject_roi # toggle this to sig_electrodes_per_subject_roi if just using sig elecs, or electrodes_per_subject_roi if using all elecs

if electrodes == all_electrodes_per_subject_roi:
    elec_string_to_add_to_filename = 'all_elecs'
elif electrodes == sig_electrodes_per_subject_roi:
    elec_string_to_add_to_filename = 'sig_elecs'
else:
    elec_string_to_add_to_filename = None

roi_labeled_arrays = put_data_in_labeled_array_per_roi_subject(
    subjects_tfr_objects,
    condition_names,
    rois,
    subjects,
    electrodes, 
    obs_axs=0,  # Trials dimension (ignoring the conditions dimension for now cuz we iterate over it)
    chans_axs=1,  # Channels dimension
    freq_axs=2,
    time_axs=3,   # Time dimension
    random_state=42  # For reproducibility
)

ROI 'lpfc': Maximum trials per condition:
  Condition 'Stimulus_bigS': 224 trials from subjects ['D0059']
  Condition 'Stimulus_bigH': 237 trials from subjects ['D0063']
in roi lpfc, subject D0057 has 220 trials for condition Stimulus_bigS
in roi lpfc, subject D0057 has 229 trials for condition Stimulus_bigH
Subject D0057, ROI lpfc, LabeledArray shape: (2, 237, 17, 99, 357)
in roi lpfc, subject D0059 has 224 trials for condition Stimulus_bigS
in roi lpfc, subject D0059 has 224 trials for condition Stimulus_bigH
Subject D0059, ROI lpfc, LabeledArray shape: (2, 237, 5, 99, 357)
in roi lpfc, subject D0063 has 215 trials for condition Stimulus_bigS
in roi lpfc, subject D0063 has 237 trials for condition Stimulus_bigH
Subject D0063, ROI lpfc, LabeledArray shape: (2, 237, 36, 99, 357)
in roi lpfc, subject D0103 has 222 trials for condition Stimulus_bigS
in roi lpfc, subject D0103 has 226 trials for condition Stimulus_bigH
Subject D0103, ROI lpfc, LabeledArray shape: (2, 237, 11, 99, 357)
ROI

In [14]:
# this is kind of unnecessary for testing with just big letter but good for the broader comparisons

# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
save_dir = os.path.join(LAB_root, 'BIDS-1.1_GlobalLocal', 'BIDS', 'derivatives', 'decoding', 'james_sun_cluster_decoding', f"{conditions_save_name}")
os.makedirs(save_dir, exist_ok=True)
print(f"Save directory created or already exists at: {save_dir}")

condition_comparisons = {}

if conditions == experiment_conditions.stimulus_experiment_conditions:
    condition_comparisons['congruency'] = [['c25', 'c75'], ['i25', 'i75']]
    condition_comparisons['switchType'] = [['r25', 'r75'], ['s25', 's75']]
    
elif conditions == experiment_conditions.stimulus_conditions:
    condition_comparisons['BigLetter'] = ['bigS', 'bigH']
    condition_comparisons['SmallLetter'] = ['smallS', 'smallH']
    condition_comparisons['Task'] = ['taskG', 'taskL']
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    condition_comparisons['BigLetter'] = ['bigS', 'bigH']
    
    # condition_comparisons['BigLetter'] = ['BigLetters', 'BigLetterh']
    # condition_comparisons['SmallLetter'] = ['SmallLetters', 'SmallLetterh']
    # condition_comparisons['Task'] = ['Taskg', 'Taskl']
    # condition_comparisons['TaskBigLetterInteraction'] = [['BigLetters/SmallLetters/Taskg', 'BigLetters/SmallLetterh/Taskg'], ['BigLetters/SmallLetters/Taskl', 'BigLetters/SmallLetterh/Taskl'], ['BigLetterh/SmallLetters/Taskg', 'BigLetterh/SmallLetterh/Taskg'], ['BigLetterh/SmallLetters/Taskl', 'BigLetterh/SmallLetterh/Taskl']]
    # condition_comparisons['TaskSmallLetterInteraction'] = [['BigLetters/SmallLetters/Taskg', 'BigLetterh/SmallLetters/Taskg'], ['BigLetters/SmallLetters/Taskl', 'BigLetterh/SmallLetters/Taskl'], ['BigLetters/SmallLetterh/Taskg', 'BigLetterh/SmallLetterh/Taskg'], ['BigLetters/SmallLetterh/Taskl', 'BigLetters/SmallLetterh/Taskl']]

Save directory created or already exists at: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\decoding\james_sun_cluster_decoding\stimulus_big_letter_conditions_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_4_subjects


In [ ]:
for condition_comparisons, strings_to_find in condition_comparisons.items():
    confusion_matrices, cats, channel_masks = get_confusion_matrix_for_rois_tfr_cluster(
        roi_labeled_arrays, rois, strings_to_find, stat_func, 
        Decoder, explained_variance=explained_variance,
        p_thresh=p_thresh, n_perm=n_perm, 
        n_splits=n_splits, n_repeats=n_repeats, obs_axs=obs_axs, chans_axs=chans_axs,
        balance_method=balance_method, oversample=oversample,
        random_state=random_state, alpha=alpha, ignore_adjacency=ignore_adjacency, seed=seed, tails=tails,
        clear_memory=clear_memory, normalize=normalize
    )

Processing ROI: lpfc
Concatenated data shape for lpfc: (474, 69, 99, 357)
Condition bigS has 204 trials
Condition bigH has 198 trials
  Repeat 1/5, Fold 1/5
stat_func returns a tuple. Taking the first element


c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\scipy\stats\_stats_py.py:1081: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\scipy\stats\_stats_py.py:1081: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   11.0s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   11.0s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   11.0s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   11.0s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:   11.1s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:   11.1s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:   11.1s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:

stat_func returns a tuple. Taking the first element


[Parallel(n_jobs=8)]: Done  99 out of  99 | elapsed:   18.8s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:    1.3s
[

stat_func returns a tuple. Taking the first element


...

stat_func returns a tuple. Taking the first element


[Parallel(n_jobs=8)]: Done  96 out of  99 | elapsed:    8.7s remaining:    0.2s
[Parallel(n_jobs=8)]: Done  99 out of  99 | elapsed:    8.8s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done  13 tasks      |

stat_func returns a tuple. Taking the first element


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:    1.7s
[Parallel(

TypeError: _ArrayMemoryError.__init__() missing 1 required positional argument: 'dtype'

In [ ]:
for roi in rois:
    file_name = f'{roi}_{conditions_save_name}_time_averaged_confusion_matrix_{n_splits}splits_{n_repeats}repeats_{balance_method}.png'
    plot_and_save_confusion_matrix(confusion_matrices[roi], cats[roi], file_name, save_dir)

In [ ]:
# This part is fine for getting the time and frequency axes
first_sub = subjects[0]
first_condition = list(subjects_tfr_objects[first_sub].keys())[0]
times = subjects_tfr_objects[first_sub][first_condition].times
freqs = subjects_tfr_objects[first_sub][first_condition].freqs
subjects_tfr_objects_save_dir = os.path.join(layout.root, 'derivatives', 'spec', spec_method, 'subjects_tfr_objects')


for roi in rois:
    # This assumes 'channel_masks' has the structure {roi: {repeat: {split: {ch_idx: mask}}}}
    if roi not in channel_masks:
        continue

    for repeat in range(n_repeats):
        if repeat not in channel_masks[roi]:
            continue
        for split in range(n_splits):
            if split not in channel_masks[roi][repeat]:
                continue
            
            # This is the dictionary of 2D masks for the current split
            mask_dict = channel_masks[roi][repeat][split]

            if not mask_dict:
                print(f"Skipping plot for {roi} Repeat {repeat+1} Split {split+1}: No masks found.")
                continue

            # --- FIX #1 & #2: Get the correct channel names and stack the masks ---
            # Get the channel indices from the dictionary keys and sort them for consistent order
            roi_ch_indices = sorted(mask_dict.keys())
            
            # Use the indices to get the corresponding channel names from the LabeledArray for this ROI
            roi_ch_names = [roi_labeled_arrays[roi].labels[chans_axs][i] for i in roi_ch_indices]
            
            # Stack the 2D mask arrays into a single 3D NumPy array
            stacked_masks = np.array([mask_dict[ch_idx] for ch_idx in roi_ch_indices])

            # --- FIX #3: Correct the title prefix ---
            title = f"{roi} Repeat {repeat+1} Split {split+1}: "

            # Now, call the plotting function with the corrected data
            roi_repeat_split_pages = plot_mask_pages(
                stacked_masks,
                roi_ch_names, # Use the ROI-specific names
                times=times,
                freqs=freqs,
                channels_per_page=60,
                grid_shape=(6, 10),
                cmap=parula_map,
                title_prefix=title, # Use the corrected title
                log_freq=True,
                show=False)

            # Save each page as a separate figure file
            for i, fig in enumerate(roi_repeat_split_pages):
                # Use the corrected filename that doesn't rely on `sub`
                fig_name = f"{roi}_sig_clusters_repeat{repeat+1}_split{split+1}_{conditions_save_name}_page_{i+1}.png"
                fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
                fig.savefig(fig_pathname, bbox_inches='tight')
                print("Saved figure:", fig_pathname)
                plt.close(fig) # Close the figure to free up memory

In [ ]:
doctest.testmod(verbose=True)